In [1]:
!pip install python-docx pandas pyyaml

In [2]:
from docx import Document

def extract_text(doc_path):
    """提取 Word 文档中的纯净文本"""
    doc = Document(doc_path)
    lines = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
    return "\n".join(lines)

doc_content = extract_text("AT_Commands.docx")
print(doc_content[:500])  # 预览前500字符

通用AT指令
ATI：获取模组厂商信息
获取模组厂商信息，包括厂家、型号和版本。
命令格式
参数
示例
AT+GMR：查询版本信息
查询软件版本信息。
命令格式
参数
示例
AT+CSQ：获取信号强度
查询接收信号强度<rssi>。
命令格式
参数
示例
AT+CREG：查询网络注册状态
查询模组的当前网络注册状态。
命令格式
参数
示例
AT+CEREG：获取EPS网络注册状态
查询EPS网络注册状态。
命令格式
参数
示例
AT+COPS：网络选择
查询网络。
命令格式
参数
示例
AT+CIMI：查询国际移动用户识别码
获取国际移动用户识别码IMSI（international mobile subscriber identification）。
命令格式
参数
示例
AT+CGSN：获取通信模组IMEI号
获取模组的产品序列号，也就是IMEI号（International Mobile Equipment Identity）。
命令格式
参数
示例
AT+GSN：获取通信模组IMEI号
获取模组的产品序列号，也就是IMEI号（International Mobile Equip


In [3]:
import re

COMMAND_PATTERN = r'^(AT[\+\w]+)[:：]\s*(.*)'  # 命令 + 描述
PARAM_PATTERN = r'<([^>]+)>\s*(.*)'            # 参数

def parse_commands(text):
    """从文档内容中提取 AT 命令结构化信息"""
    lines = text.splitlines()
    commands, current = [], None
    section = None

    for line in lines:
        # 检测命令开始
        cmd_match = re.match(COMMAND_PATTERN, line)
        if cmd_match:
            if current:
                commands.append(current)
            current = {
                "command": cmd_match.group(1),
                "description": cmd_match.group(2).strip(),
                "format": "",
                "parameters": [],
                "examples": []
            }
            section = None
            continue

        if not current:
            continue

        # 检测小节
        if line.startswith("命令格式"):
            section = "format"
            continue
        elif line.startswith("参数"):
            section = "parameters"
            continue
        elif line.startswith("示例"):
            section = "examples"
            continue

        # 收集内容
        if section == "format":
            current["format"] += line + " "
        elif section == "parameters":
            param_match = re.match(PARAM_PATTERN, line)
            if param_match:
                current["parameters"].append({
                    "name": param_match.group(1),
                    "desc": param_match.group(2)
                })
        elif section == "examples":
            current["examples"].append(line)

    if current:
        commands.append(current)
    return commands

In [4]:
def clean_commands(commands):
    for cmd in commands:
        cmd["format"] = cmd["format"].strip()
        cmd["examples"] = [ex.strip() for ex in cmd["examples"] if ex.strip()]
    return commands

In [5]:
import pandas as pd

def export_csv(commands, path="at_commands.csv"):
    rows = []
    for cmd in commands:
        rows.append({
            "command": cmd["command"],
            "description": cmd["description"],
            "format": cmd["format"],
            "parameters": "; ".join([f"{p['name']}:{p['desc']}" for p in cmd["parameters"]]),
            "examples": "; ".join(cmd["examples"])
        })
    df = pd.DataFrame(rows)
    df.to_csv(path, index=False, encoding="utf-8-sig")
    return df


In [6]:
import yaml

def export_yaml(commands, path="at_commands.yaml"):
    with open(path, "w", encoding="utf-8") as f:
        yaml.dump({"at_commands": commands}, f, allow_unicode=True, sort_keys=False)

In [7]:
def validate(commands):
    issues = []
    seen = set()
    for cmd in commands:
        if not cmd["description"]:
            issues.append(f"{cmd['command']} 缺少描述")
        if cmd["command"] in seen:
            issues.append(f"{cmd['command']} 出现重复")
        seen.add(cmd["command"])
    return issues


In [8]:
doc_content = extract_text("AT_Commands.docx")
commands = parse_commands(doc_content)
commands = clean_commands(commands)

# 导出
df = export_csv(commands, "at_commands.csv")
export_yaml(commands, "at_commands.yaml")

# 验证
issues = validate(commands)
print("发现问题:", issues)


发现问题: []
